## Transformação camada prata: product

In [0]:
%run ../Config/DeltaFunctions

In [0]:
%run ../Config/LogProcessamento

In [0]:
from pyspark.sql import DataFrame, Window
from pyspark.sql import functions as F
from pyspark.sql.types import (
    IntegerType, StringType, TimestampType, StructType, StructField, BooleanType
)

In [0]:
# Habilitar a evolução automática de esquemas
spark.sql("SET spark.databricks.delta.schema.autoMerge.enabled = true")
spark.sql('USE CATALOG hive_metastore')

# Informações da Tabela Fonte
source_table = "production_product"
source_database = "adventure_works_bronze"
bronze_source_table = spark.read.table(f"{source_database}.{source_table}")

# Informações da Tabela Destino (target)
target_table_name = "production_product"
target_database = "adventure_works_silver"
target_table = f"{target_database}.{target_table_name}"

primary_keys = ["ProductID"]

In [0]:
expected_schema = StructType([
    StructField("ProductID", IntegerType(), False),                  # int IDENTITY(1,1) NOT NULL
    StructField("Name", StringType(), False),                        # Name NOT NULL
    StructField("ProductNumber", StringType(), False),               # nvarchar(25) NOT NULL
    StructField("MakeFlag", BooleanType(), False),                   # Flag NOT NULL
    StructField("FinishedGoodsFlag", BooleanType(), False),          # Flag NOT NULL
    StructField("Color", StringType(), True),                        # nvarchar(15) NULL
    StructField("SafetyStockLevel", ShortType(), False),             # smallint NOT NULL
    StructField("ReorderPoint", ShortType(), False),                 # smallint NOT NULL
    StructField("StandardCost", DecimalType(19, 4), False),          # money NOT NULL
    StructField("ListPrice", DecimalType(19, 4), False),             # money NOT NULL
    StructField("Size", StringType(), True),                         # nvarchar(5) NULL
    StructField("SizeUnitMeasureCode", StringType(), True),          # nchar(3) NULL
    StructField("WeightUnitMeasureCode", StringType(), True),        # nchar(3) NULL
    StructField("Weight", DecimalType(8, 2), True),                  # decimal(8, 2) NULL
    StructField("DaysToManufacture", IntegerType(), False),          # int NOT NULL
    StructField("ProductLine", StringType(), True),                  # nchar(2) NULL
    StructField("Class", StringType(), True),                        # nchar(2) NULL
    StructField("Style", StringType(), True),                        # nchar(2) NULL
    StructField("ProductSubcategoryID", IntegerType(), True),        # int NULL
    StructField("ProductModelID", IntegerType(), True),              # int NULL
    StructField("SellStartDate", TimestampType(), False),            # datetime NOT NULL
    StructField("SellEndDate", TimestampType(), True),               # datetime NULL
    StructField("DiscontinuedDate", TimestampType(), True),          # datetime NULL
    StructField("rowguid", StringType(), False),                     # uniqueidentifier NOT NULL
    StructField("ModifiedDate", TimestampType(), False)              # datetime NOT NULL
])


In [0]:
def transform_production_product(Product: DataFrame) -> DataFrame:
    '''
    Transformação da tabela: product
    Parâmetros:
        product (DataFrame): DataFrame contendo os dados da tabela product

    Retorna:
        DataFrame: O DataFrame resultante após a transformação e deduplicação.
    '''
    # Define valores padrão para campos que podem ser nulos
    Product = Product.withColumn(
        'rowguid',
        F.when(F.col('rowguid').isNull(), F.expr('uuid()')).otherwise(F.col('rowguid'))
    )
    
    Product = Product.withColumn(
        'ModifiedDate',
        F.when(F.col('ModifiedDate').isNull(), F.current_timestamp()).otherwise(F.col('ModifiedDate'))
    )
    
    # # Aplicando checks de integridade
    # Product = Product.filter(F.col('SafetyStockLevel') > 0)
    # Product = Product.filter(F.col('ReorderPoint') > 0)
    # Product = Product.filter(F.col('StandardCost') >= 0.00)
    # Product = Product.filter(F.col('ListPrice') >= 0.00)
    # Product = Product.filter(F.col('DaysToManufacture') >= 0)
    # Product = Product.filter((F.upper(F.col('Class')).isin('H', 'M', 'L')) | F.col('Class').isNull())
    # Product = Product.filter((F.upper(F.col('ProductLine')).isin('R', 'M', 'T', 'S')) | F.col('ProductLine').isNull())
    # Product = Product.filter((F.upper(F.col('Style')).isin('U', 'M', 'W')) | F.col('Style').isNull())
    # Product = Product.filter((F.col('Weight') > 0.00) | F.col('Weight').isNull())
    # Product = Product.filter((F.col('SellEndDate') >= F.col('SellStartDate')) | F.col('SellEndDate').isNull())
    
    # Define a função de janela para deduplicar com base nas chaves primárias
    window_spec = Window.partitionBy('ProductID').orderBy(F.col('ModifiedDate').desc())
    Product = Product.withColumn('row_num', F.row_number().over(window_spec))

    # Filtra para manter apenas a primeira linha em cada partição (sem duplicatas)
    Product = Product.filter(F.col('row_num') == 1).drop('row_num')

    # Seleção final com CAST explícito dos tipos de dados
    Product = Product.select(
        F.col('ProductID').cast(IntegerType()).alias('ProductID'),
        F.col('Name').cast(StringType()).alias('Name'),
        F.col('ProductNumber').cast(StringType()).alias('ProductNumber'),
        F.col('MakeFlag').cast(BooleanType()).alias('MakeFlag'),
        F.col('FinishedGoodsFlag').cast(BooleanType()).alias('FinishedGoodsFlag'),
        F.col('Color').cast(StringType()).alias('Color'),
        F.col('SafetyStockLevel').cast(ShortType()).alias('SafetyStockLevel'),
        F.col('ReorderPoint').cast(ShortType()).alias('ReorderPoint'),
        F.col('StandardCost').cast(DecimalType(19, 4)).alias('StandardCost'),
        F.col('ListPrice').cast(DecimalType(19, 4)).alias('ListPrice'),
        F.col('Size').cast(StringType()).alias('Size'),
        F.col('SizeUnitMeasureCode').cast(StringType()).alias('SizeUnitMeasureCode'),
        F.col('WeightUnitMeasureCode').cast(StringType()).alias('WeightUnitMeasureCode'),
        F.col('Weight').cast(DecimalType(8, 2)).alias('Weight'),
        F.col('DaysToManufacture').cast(IntegerType()).alias('DaysToManufacture'),
        F.col('ProductLine').cast(StringType()).alias('ProductLine'),
        F.col('Class').cast(StringType()).alias('Class'),
        F.col('Style').cast(StringType()).alias('Style'),
        F.col('ProductSubcategoryID').cast(IntegerType()).alias('ProductSubcategoryID'),
        F.col('ProductModelID').cast(IntegerType()).alias('ProductModelID'),
        F.col('SellStartDate').cast(TimestampType()).alias('SellStartDate'),
        F.col('SellEndDate').cast(TimestampType()).alias('SellEndDate'),
        F.col('DiscontinuedDate').cast(TimestampType()).alias('DiscontinuedDate'),
        F.col('rowguid').cast(StringType()).alias('rowguid'),
        F.col('ModifiedDate').cast(TimestampType()).alias('ModifiedDate')
    )

    return Product


## Aplicar Transformação

In [0]:
# Estrutura do log para registrar informações sobre o processo
log_data = {
    "log_tabela": source_table,
    "log_camada": "Silver",
    "log_origem": "adventure_works_bronze",
    "log_destino": "adventure_works_silver",
}

# Registra o início do processo
addlog(**log_data, log_status='Início', atualizacao=0)

try:
    # Realiza a transformação dos dados
    transformed_df = transform_production_product(Product=bronze_source_table)

    # Verifica rapidamente o número de linhas e o schema do DataFrame
    row_count = transformed_df.count()
    transformed_df.printSchema()

    # Validação do schema
    is_schema_valid = _validate_schema(transformed_df, expected_schema)
    if is_schema_valid:
        addlog(**log_data, log_status='Sucesso', atualizacao=1)
        print("O schema do DataFrame está correto.")
    else:
        raise ValueError("Schema validation failed.")
    
except Exception as e:
    # Registra erro caso ocorra uma exceção
    addlog(**log_data, log_status='Falha', atualizacao=1)
    print(f"Erro ao processar a tabela: {str(e)}")
    raise  

# Se o schema for válido, realiza o upsert
_upsert_silver_table(transformed_df, target_table, primary_keys, not_matched_by_source_action="DELETE")
